In [2]:
s3_path = "s3://sagemaker-automated-execution-034700280673-us-east-1/data_sample/"

In [3]:
import pandas as pd
import json

jsonl_file_name = "dataset.josnl"
with open(
    jsonl_file_name,
    "a",
) as outfile:
    for i in range(1, 18):
        print(i)
        file_name = f"dataset_1_{i}.jsonl"
        df = pd.read_csv(s3_path + file_name + ".csv", header=None)
        col2 = df.iloc[:, 1]
        for value in col2:
            d = value.replace("content': \"", "content': '")
            d = d.replace('"}, {', "'}, {")
            d = d.replace('"}]', "'}]")
            d = d.replace('"', "")
            # d=d.replace("'","\"")
            d = d.replace("'messages'", '"messages"')
            d = d.replace(
                "'role': 'assistant', 'content': '", '"role": "assistant", "content": "'
            )
            d = d.replace(
                "'role': 'user', 'content': '", '"role": "user", "content": "'
            )
            d = d.replace("'}, {", '"}, {')
            d = d.replace("'}]", '"}]')
            d = d.replace("\\'", "'")
            try:
                dl = json.loads(d)
                json.dump(dl, outfile)
                outfile.write("\n")
            except Exception as e:
                print(e)
                print(f"value:{value}")
                print(f"d:{d}")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [4]:
import openai
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="env")

openai.api_key = os.getenv("OPENAI_API_KEY")

openai.File.create(
  file=open("dataset.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-sTMlXQOhnbrf2BPOar0QGhaC at 0x7f43240e5300> JSON: {
  "object": "file",
  "id": "file-sTMlXQOhnbrf2BPOar0QGhaC",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 26845831,
  "created_at": 1693800940,
  "status": "uploaded",
  "status_details": null
}

In [15]:
openai.File.list()

<OpenAIObject list at 0x7f4322e31e90> JSON: {
  "object": "list",
  "data": [
    {
      "object": "file",
      "id": "file-XEfiILR0iGtbUdb6AhC5jAxO",
      "purpose": "fine-tune",
      "filename": "dataset_ft.jsonl",
      "bytes": 2965,
      "created_at": 1692900787,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-yAVB3yIy5HR3wReWvivUP1VK",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 26845831,
      "created_at": 1693800816,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-2McG142K9ZccG4SsReINWlDa",
      "purpose": "fine-tune",
      "filename": "dataset_ft.jsonl",
      "bytes": 2146,
      "created_at": 1692900663,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-u9QjmgJucuBpEzYdYMdPdv1P",
      "purpose": "fine-tune-results",
      "filename": "step_metrics.cs

In [9]:
openai.FineTuningJob.create(training_file="file-sTMlXQOhnbrf2BPOar0QGhaC", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-VwULaBX0VbLZpHa63IOqDQCI at 0x7f4322c9bb00> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-VwULaBX0VbLZpHa63IOqDQCI",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693801464,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-2fsrzAoUw9bQrG5WjnmdMPmz",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-sTMlXQOhnbrf2BPOar0QGhaC",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [11]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x7f4322e33240> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-VwULaBX0VbLZpHa63IOqDQCI",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693801464,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-2fsrzAoUw9bQrG5WjnmdMPmz",
      "result_files": [],
      "status": "running",
      "validation_file": null,
      "training_file": "file-sTMlXQOhnbrf2BPOar0QGhaC",
      "hyperparameters": {
        "n_epochs": 3
      },
      "trained_tokens": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-pii8qXPoVNZpzEeMIoaOTCOs",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1692900804,
      "finished_at": 1692901199,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:swft-blockchain::7r9Dw0NR",
      "organization_id": "org-2fsrzAoUw9bQrG5WjnmdMPmz",
      "result_files": [
        "file-u9QjmgJucuBpEzYdYMdPdv1P"
      ],
      "status": "suc

In [17]:
# Retrieve the state of a fine-tune
openai.FineTuningJob.retrieve("ftjob-VwULaBX0VbLZpHa63IOqDQCI")

<FineTuningJob fine_tuning.job id=ftjob-VwULaBX0VbLZpHa63IOqDQCI at 0x7f4322e092b0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-VwULaBX0VbLZpHa63IOqDQCI",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693801464,
  "finished_at": 1693809328,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:swft-blockchain::7uxTFmf0",
  "organization_id": "org-2fsrzAoUw9bQrG5WjnmdMPmz",
  "result_files": [
    "file-lqkS2qwENKDq8zQVfrhhydxx"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-sTMlXQOhnbrf2BPOar0QGhaC",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 15490221
}

In [ ]:
# Cancel a job
openai.FineTuningJob.cancel("ft-abc123")

# List up to 10 events from a fine-tuning job
openai.FineTuningJob.list_events(id="ft-abc123", limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
openai.Model.delete("ft-abc123")

In [ ]:
questions = []
for i in range(1, 18):
    q_file_name = f"dataset_1_{i}_n_q.csv"
    q_csv = pd.read_csv(s3_path + q_file_name)
    for q in q_csv["questions"]:
        questions.append(q)
df = pd.DataFrame()
df["question"] = pd.Series(questions)
df.to_csv("questions.csv")